In [2]:
from neo4j.v1 import GraphDatabase
import numpy as np

In [4]:
driver = GraphDatabase.driver('bolt://localhost:7687',auth=('neo4j', 'password'))


In [5]:
def print_results(array):
    a = np.array(array)
    print('Average response: {}ms'.format(a.mean()))
    print('Max response: {}ms'.format(max(a)))
    print('Min response: {}ms'.format(min(a)))
    for p in [25,50,75,90,95,99]:
        print('{}th percentile: {}ms'.format(p, np.percentile(a,p)))
        
def test_query(query):
    session = driver.session()
    results = []
    for _ in range(1000):
        r = session.run(query)
        summary = r.summary()
        millis = summary.result_available_after + summary.result_consumed_after
        results.append(millis)
    session.close()
    print_results(results)
    
def import_query(query):
    with driver.session() as session:
        session.run(query)
        session.run("call apoc.warmup.run()")

def delete_db_query():
    delete_query = """
    CALL apoc.periodic.commit('MATCH (n) WITH n limit $limit 
                           DETACH DELETE n RETURN count(*)', 
                          {limit:100000})
    """
    with driver.session() as session:
        session.run(delete_query)

# First example with reaction type as attribute

## Import

In [7]:
import_as_attribute = """

CALL apoc.periodic.iterate(
  "UNWIND range(0,100) as i
   return i",
  "CREATE (post:Post{id:i})
   WITH i,post
   UNWIND range (0,10000) as j
   WITH i,j,post, CASE j%6 WHEN 0 THEN 'like'
                           WHEN 1 THEN 'love'
                           WHEN 2 THEN 'haha'
                           WHEN 3 THEN 'wow'
                           WHEN 4 THEN 'sad'
                           WHEN 5 THEN 'angry' END as reaction
   CREATE (post)<-[:REACTION{type:reaction}]-(:User)",
  {parallel:True})

"""

import_query(import_as_attribute)

## Load test

In [8]:
attribute_first_query = """
MATCH (p:Post)<-[rel]-()
RETURN p.id as id, rel.type as type, count(*) AS count
ORDER by count DESC limit 5
"""
test_query(attribute_first_query)

Average response: 799.739ms
Max response: 1860ms
Min response: 768ms
25th percentile: 783.0ms
50th percentile: 787.0ms
75th percentile: 792.0ms
90th percentile: 806.0ms
95th percentile: 860.0999999999999ms
99th percentile: 1071.03ms


In [9]:
attribute_second_query = """
MATCH (p:Post)<-[rel]-()
WITH p, rel.type as type, count(*) AS count
ORDER by count DESC limit 5
RETURN p.id,type,count

"""
test_query(attribute_second_query)

Average response: 657.88ms
Max response: 908ms
Min response: 644ms
25th percentile: 650.0ms
50th percentile: 653.0ms
75th percentile: 657.0ms
90th percentile: 667.0ms
95th percentile: 687.0ms
99th percentile: 760.1099999999999ms


# Second example with reaction type as node label

In [10]:
delete_db_query()

## Import

In [11]:
import_as_node_label = """
CALL apoc.periodic.iterate(
  "UNWIND range(0,100) as i
   return i",
  "CREATE (post:Post{id:i})
   with i,post
   UNWIND range(0,10000) as j
   WITH i,post, CASE j%6 WHEN 0 THEN 'like'
   WHEN 1 THEN 'love'
   WHEN 2 THEN 'haha'
   WHEN 3 THEN 'wow'
   WHEN 4 THEN 'sad'
   WHEN 5 THEN 'angry' END as reaction
   // Create node with a dynamic label
   call apoc.create.node([reaction], {}) yield node
   CREATE (post)<-[:TO]-(node)
   CREATE (node)<-[:REACTION]-(:User)",
  {parallel:True})
"""
import_query(import_as_node_label)

## Load test

In [12]:
label_query = """
MATCH (p:Post)<--(reaction)
WITH p,labels(reaction) as labels,count(*) as count
ORDER BY count DESC LIMIT 5
RETURN p.id as id,labels[0] as type,count(*)

"""
test_query(label_query)

Average response: 569.038ms
Max response: 1255ms
Min response: 548ms
25th percentile: 553.0ms
50th percentile: 556.0ms
75th percentile: 560.0ms
90th percentile: 570.1ms
95th percentile: 637.0999999999999ms
99th percentile: 903.3099999999997ms


# Third example with reaction type as relationship type

In [16]:
delete_db_query()

## Import

In [17]:
import_type_as_rel_type = """
CALL apoc.periodic.iterate(
  "UNWIND range(0,100) as i
   return i",
  "CREATE (post:Post{id:i})
   with i,post
   UNWIND range(0,10000) as j
   CREATE (u:User)
   WITH i,post,u, CASE j%6 WHEN 0 THEN 'like'
                           WHEN 1 THEN 'love'
                           WHEN 2 THEN 'haha'
                           WHEN 3 THEN 'wow'
                           WHEN 4 THEN 'sad'
                           WHEN 5 THEN 'angry' END as reaction
  // Create relationship with a dynamic type
  CALL apoc.create.relationship(u, reaction, {}, post) YIELD rel
  RETURN count(*)",
  {parallel:True})
"""
import_query(import_type_as_rel_type)

## Load test

In [18]:
rel_type_first_query = """
MATCH (p:Post)<-[rel]-()
WITH p,type(rel) as type,count(*) as count
ORDER BY count DESC limit 5 
RETURN p.id as id,type,count

"""
test_query(rel_type_first_query)

Average response: 629.604ms
Max response: 903ms
Min response: 607ms
25th percentile: 616.0ms
50th percentile: 628.0ms
75th percentile: 635.0ms
90th percentile: 648.1ms
95th percentile: 656.0ms
99th percentile: 698.0699999999999ms


In [19]:
rel_type_second_query = """
MATCH (p:Post)
WITH p
UNWIND [{key:'like',count:size((p)<-[:like]-())},
        {key:'love',count:size((p)<-[:love]-())},
        {key:'haha',count:size((p)<-[:haha]-())},
        {key:'wow', count: size((p)<-[:wow]-())},
        {key:'sad', count: size((p)<-[:sad]-())},
        {key:'angry',count: size((p)<-[:angry]-())}] as k
WITH p,k.key as key,k.count as count
ORDER BY count DESC LIMIT 5
RETURN p.id as id,key,count

"""
test_query(rel_type_second_query)

Average response: 0.998ms
Max response: 82ms
Min response: 0ms
25th percentile: 1.0ms
50th percentile: 1.0ms
75th percentile: 1.0ms
90th percentile: 1.0ms
95th percentile: 2.0ms
99th percentile: 2.0ms


In [20]:
rel_type_apoc_query = """

MATCH (p:Post)
UNWIND apoc.node.relationship.types(p) as type
WITH p, type, apoc.node.degree.in(p, type) as count
ORDER BY count DESC LIMIT 5
RETURN p.id as post, type, count

"""
test_query(rel_type_apoc_query)

Average response: 1.869ms
Max response: 74ms
Min response: 1ms
25th percentile: 1.0ms
50th percentile: 2.0ms
75th percentile: 2.0ms
90th percentile: 3.0ms
95th percentile: 3.0ms
99th percentile: 4.0ms
